#### Email simples

In [1]:
# PESQUISAR
# https://realpython.com/python-send-email/

# Importação das bibliotecas
import pandas as pd
import win32com.client as win32
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

In [3]:
# 
var1 = "SORRIA"
mail.To = "daniel_ijano@hotmail.com"
mail.Subject = "Assunto"
mail.HTMLBody = f"""
<p>Bodia raio disol
{var1}
<p>:)
"""

# anexo = "C://Users/blablabla/arquivo.file"
# email.Attachments.Add(anexo)

#mail.Send()

#### Iterando

In [4]:
# Create the connection with Google Cloud
def create_connection():
        # Set the credentials and create the connection
        key_path = "../usp-mba-dsa-tcc-4277103d9155.json"

        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/bigquery"]
        )

        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id,
        )
        return client

client = create_connection()

In [36]:
# Perform a query.
QUERY = (
    "SELECT * FROM `usp-mba-dsa-tcc.ecommerce_offers.vw_send_alerts` WHERE Status='Active'"
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = rows.to_dataframe()

In [37]:
df.head(5)

,Alert_id,User_id,Product_id,Target,Created_at,Status,user_name,Email,regular_price,sub_price,Loja,Marca,Descricao,URL,img
0,00001,00001,t3kqJ57KOg5qIffC/S571w+DxRzWqt62lOVKcRz3B0w=,180.0,2023-09-19,Active,Teste Hotmail,daniel_ijano@hotmail.com,189.90,170.91,Cobasi,Magnus,Ração Magnus Chips Cães Adultos Carne e Frango...,https://www.cobasi.com.br/racao-magnus-chips-c...,None
1,00003,00002,4immtn7eSCx3nAT4wkBLvI/aeCqOi4CrhjgDliz5NaQ=,70.0,2023-09-19,Active,Teste Gmail,daniel.ijano@gmail.com,75.90,68.31,Cobasi,Premier Pet,Ração Premier Nattu Gatos Adultos Castrados Ab...,https://www.cobasi.com.br/racao-premier-nattu-...,None
2,00004,00002,lCJkFl8oZbIcaeKd5NlTn/2NI6Un6VIEbInuiQBcHhI=,190.0,2023-09-19,Active,Teste Gmail,daniel.ijano@gmail.com,223.11,200.80,Petlove,Alinutri,Ração Seca Alinutri Nature Fórmula Pro para Ga...,https://www.petlove.com.br/racao-seca-alinutri...,None


In [35]:
outlook = win32.Dispatch('outlook.application')

for row in df.index[:1]:#TEMP
    if float(df['sub_price'][row]) <= float(df['Target'][row]):

        mail = outlook.CreateItem(0)
        mail.To = df['Email'][row]
        mail.Subject = "Seu Alerta de Preço Foi Atingido!"
        mail.HTMLBody = f"""
        <p style="font-family: sans-serif; color: #4c4c4c;">Olá {df['user_name'][row]}<br>O produto que você estava monitorando baixou!!</p>

        <div class="container" style="display: flex; align-items: center; max-width: 500px">
          <img class="image" src={df['img'][row]} alt="Imagem do Produto" width=200 height=auto>
          <strong><p style="font-family: sans-serif; font-size: 20px; color: #4c4c4c;">{df['Marca'][row]}</p>
          <p style="font-family: sans-serif; font-size: 16px; color: #4c4c4c;">{df['Descricao'][row]}</p>
          <p style="font-family: sans-serif; font-size: 24px; color: #4c4c4c;">R${str("{:.2f}".format(df['sub_price'][row])).replace('.', ',')}</p></strong>
        </div>
        
        <br><strong><a href={df['URL'][row]} style="display: block; text-align: center; background-color: #8ed7c1; padding: 10px; 
        font-family: sans-serif; font-size: 16px; max-width: 500px; text-decoration: none; color: #262626;">IR PARA {df['Loja'][row].upper()}</a></strong>
        """
        # https://html.onlineviewer.net/
        # https://htmlcheatsheet.com/
        mail.Send()
# https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/